 I'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle. We've made a few modifications from the original dataset that was uploaded to Kaggle:

We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
We dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

In [194]:
import pandas as pd
import numpy as np
import re

autos = pd.read_csv("autos.csv", encoding="Latin-1")
print(autos.head())


           dateCrawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offerType   price   abtest vehicleType  yearOfRegistration  \
0  privat   Angebot  $5,000  control         bus                2004   
1  privat   Angebot  $8,500  control   limousine                1997   
2  privat   Angebot  $8,990     test   limousine                2009   
3  privat   Angebot  $4,350  control  kleinwagen                2007   
4  privat   Angebot  $1,350     test       kombi                2003   

     gearbox  powerPS   model   odometer  monthOfRegistration fuelType  \
0    manuell      158  andere 

In [195]:
print(autos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

Todas las siguientes variables tienen null values. 

vehicleType            44905 non-null object
gearbox                47320 non-null object
model                  47242 non-null object
fuelType               45518 non-null object
notRepairedDamage      40171 non-null object

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [196]:
copy_data = autos.copy()

def clean_col(col):
    col = col.replace("yearOfRegistration","registration_year")
    col = col.replace("monthOfRegistration","registration_month")
    col = col.replace("notRepairedDamage","unrepaired_damage")
    col = col.replace("dateCreated","ad_created")
    col = col.replace("odometer","odometer_km")
    return col

def camel_to_snake(col):
    col = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', col)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', col).lower()

new_columns = []
for c in copy_data.columns:
    clean_c = clean_col(c)
    new_columns.append(clean_c)

neww_columns = []
for c in new_columns:
    camel_case = camel_to_snake(c)
    neww_columns.append(camel_case)

copy_data.columns = neww_columns



In [197]:
copy_data.describe()
print(copy_data["price"][20])
print(copy_data["odometer_km"][20])


copy_data["price"] = copy_data["price"].str.replace('$','').str.replace(',','').astype(int)
copy_data["odometer_km"] = copy_data["odometer_km"].str.replace('km','').str.replace(',','').astype(int)
                                                                   
print(copy_data["price"][20])
print(copy_data["odometer_km"][20])




$3,500
150,000km
3500
150000


nr_of_pictures couldnt be 0 in all cells

In [198]:
copy_data["price"].value_counts().head()

0       1421
500      781
1500     734
2500     643
1000     639
Name: price, dtype: int64

In [199]:
copy_data.describe()

,price,registration_year,power_ps,odometer_km,registration_month,nr_of_pictures,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,9.840044e+03,2005.073280,116.355920,125732.700000,5.723360,0.0,50813.627300
std,4.811044e+05,105.712813,209.216627,40042.211706,3.711984,0.0,25779.747957
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1.100000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30451.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49577.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71540.000000
max,1.000000e+08,9999.000000,17700.000000,150000.000000,12.000000,0.0,99998.000000


Removing all cell where price is 0. 

In [200]:
new_data = copy_data[copy_data['price'] > 100000]

new_data.describe()

,price,registration_year,power_ps,odometer_km,registration_month,nr_of_pictures,postal_code
count,5.300000e+01,53.000000,53.000000,53.000000,53.000000,53.0,53.000000
mean,4.013343e+06,2002.981132,323.301887,60849.056604,5.018868,0.0,57969.245283
std,1.435784e+07,15.382921,198.013654,56173.972972,3.213497,0.0,27073.972735
min,1.049000e+05,1951.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1.290000e+05,1998.000000,111.000000,20000.000000,3.000000,0.0,42655.000000
50%,1.800000e+05,2010.000000,400.000000,30000.000000,5.000000,0.0,61462.000000
75%,9.999900e+05,2014.000000,460.000000,125000.000000,7.000000,0.0,74078.000000
max,1.000000e+08,2018.000000,799.000000,150000.000000,12.000000,0.0,99880.000000
